In [46]:
import os
import re
import warnings
from datetime import datetime

import numpy as np
import pandas as pd
from IPython.display import display
from scipy.stats import ks_2samp
from sklearn.covariance import MinCovDet
from sklearn.ensemble import IsolationForest
from sklearn.impute import KNNImputer
from sklearn.neighbors import LocalOutlierFactor
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [47]:
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
np.random.seed(42)
print(f"📅 Data preprocessing started on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

📅 Data preprocessing started on: 2026-01-28 23:39:45


In [48]:
def _safe_numeric_frame(df_in, cols):
    X = df_in[cols].copy()
    X = X.replace([np.inf, -np.inf], np.nan)
    for c in cols:
        X[c] = pd.to_numeric(X[c], errors='coerce')
    X = X.fillna(X.median(numeric_only=True))
    return X

def psi_numeric(expected, actual, bins=10):
    expected = pd.Series(expected).dropna().astype(float)
    actual = pd.Series(actual).dropna().astype(float)
    if len(expected) < 2 or len(actual) < 2:
        return np.nan
    quantiles = np.linspace(0, 1, bins + 1)
    edges = expected.quantile(quantiles).values
    edges = np.unique(edges)
    if len(edges) < 3:
        return np.nan
    expected_bins = pd.cut(expected, bins=edges, include_lowest=True)
    actual_bins = pd.cut(actual, bins=edges, include_lowest=True)
    expected_dist = expected_bins.value_counts(normalize=True).sort_index()
    actual_dist = actual_bins.value_counts(normalize=True).sort_index()
    eps = 1e-12
    expected_dist = expected_dist.clip(eps, 1)
    actual_dist = actual_dist.clip(eps, 1)
    return float(((actual_dist - expected_dist) * np.log(actual_dist / expected_dist)).sum())

def correlation_prune(df_in, cols, threshold=0.95):
    X = df_in[cols].copy()
    X = X.replace([np.inf, -np.inf], np.nan)
    X = X.fillna(X.median(numeric_only=True))
    corr = X.corr().abs()
    upper = corr.where(np.triu(np.ones(corr.shape), k=1).astype(bool))
    drop = [column for column in upper.columns if any(upper[column] > threshold)]
    return drop

def compute_vif(df_in, cols, max_features=40):
    cols = [c for c in cols if c in df_in.columns]
    cols = [c for c in cols if pd.api.types.is_numeric_dtype(df_in[c])]
    if len(cols) < 2:
        return pd.DataFrame(columns=['feature', 'vif'])
    if len(cols) > max_features:
        variances = df_in[cols].var().sort_values(ascending=False)
        cols = list(variances.head(max_features).index)
    X = df_in[cols].copy()
    X = X.replace([np.inf, -np.inf], np.nan)
    X = X.fillna(X.median(numeric_only=True))
    X = (X - X.mean()) / (X.std(ddof=0) + 1e-12)
    vifs = []
    X_np = X.values
    for i, feature in enumerate(cols):
        y = X_np[:, i]
        X_other = np.delete(X_np, i, axis=1)
        if X_other.shape[1] == 0:
            vifs.append((feature, 1.0))
            continue
        beta, *_ = np.linalg.lstsq(X_other, y, rcond=None)
        y_hat = X_other @ beta
        ss_res = float(np.sum((y - y_hat) ** 2))
        ss_tot = float(np.sum((y - np.mean(y)) ** 2))
        r2 = 1 - ss_res / (ss_tot + 1e-12)
        vif = 1.0 / (1.0 - r2 + 1e-12)
        vifs.append((feature, float(vif)))
    return pd.DataFrame(vifs, columns=['feature', 'vif']).sort_values('vif', ascending=False).reset_index(drop=True)

def mahalanobis_distance(df_in, cols, random_state=42):
    X = _safe_numeric_frame(df_in, cols)
    if X.shape[1] < 2 or X.shape[0] < 5:
        return pd.Series(np.nan, index=df_in.index)
    try:
        mcd = MinCovDet(random_state=random_state, support_fraction=None)
        mcd.fit(X.values)
        diff = X.values - mcd.location_
        inv_cov = np.linalg.inv(mcd.covariance_)
        d2 = np.einsum('ij,jk,ik->i', diff, inv_cov, diff)
        return pd.Series(np.sqrt(np.maximum(d2, 0)), index=df_in.index)
    except Exception:
        cov = np.cov(X.values, rowvar=False)
        cov = cov + np.eye(cov.shape[0]) * 1e-9
        inv_cov = np.linalg.inv(cov)
        diff = X.values - np.mean(X.values, axis=0)
        d2 = np.einsum('ij,jk,ik->i', diff, inv_cov, diff)
        return pd.Series(np.sqrt(np.maximum(d2, 0)), index=df_in.index)

In [49]:
data_path = '../data/raw/dataset.csv'
output_path = '../data/processed/'
try:
    df_raw = pd.read_csv(data_path)
    print("✅ Raw dataset loaded successfully!")
    print(f"📊 Original dataset shape: {df_raw.shape[0]:,} rows × {df_raw.shape[1]} columns")
    print(f"💾 Memory usage: {df_raw.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
except FileNotFoundError:
    print("❌ Dataset file not found. Please check the file path.")
except Exception as e:
    print(f"❌ Error loading dataset: {e}")
df = df_raw.copy()
print(f"\n🔄 Working copy created for preprocessing")

✅ Raw dataset loaded successfully!
📊 Original dataset shape: 503 rows × 15 columns
💾 Memory usage: 1.04 MB

🔄 Working copy created for preprocessing


In [50]:
print("🔍 INITIAL DATA ASSESSMENT")
print("=" * 50)
print(f"Dataset Shape: {df.shape}")
print(f"\nColumn Information:")
for i, col in enumerate(df.columns, 1):
    missing_count = df[col].isnull().sum()
    missing_pct = (missing_count / len(df)) * 100
    print(f"{i:2d}. {col:<25} | Missing: {missing_count:3d} ({missing_pct:5.1f}%)")
print(f"\n📊 Total Missing Values: {df.isnull().sum().sum():,}")
print(f"📊 Data Completeness: {((df.shape[0] * df.shape[1] - df.isnull().sum().sum()) / (df.shape[0] * df.shape[1]) * 100):.1f}%")

🔍 INITIAL DATA ASSESSMENT
Dataset Shape: (503, 15)

Column Information:
 1. Symbol                    | Missing:   0 (  0.0%)
 2. Name                      | Missing:   0 (  0.0%)
 3. Address                   | Missing:   1 (  0.2%)
 4. Sector                    | Missing:   1 (  0.2%)
 5. Industry                  | Missing:   1 (  0.2%)
 6. Full Time Employees       | Missing:   5 (  1.0%)
 7. Description               | Missing:   1 (  0.2%)
 8. Total ESG Risk score      | Missing:  73 ( 14.5%)
 9. Environment Risk Score    | Missing:  73 ( 14.5%)
10. Governance Risk Score     | Missing:  73 ( 14.5%)
11. Social Risk Score         | Missing:  73 ( 14.5%)
12. Controversy Level         | Missing:  73 ( 14.5%)
13. Controversy Score         | Missing: 100 ( 19.9%)
14. ESG Risk Percentile       | Missing:  73 ( 14.5%)
15. ESG Risk Level            | Missing:  73 ( 14.5%)

📊 Total Missing Values: 620
📊 Data Completeness: 91.8%


In [51]:
print("🧹 STEP 1: BASIC DATA CLEANING")
print("=" * 40)
initial_shape = df.shape
duplicate_rows = df.duplicated().sum()
print(f"Duplicate rows found: {duplicate_rows}")
if duplicate_rows > 0:
    df = df.drop_duplicates()
    print(f"✅ Removed {duplicate_rows} duplicate rows")
print(f"\n📋 Company Symbol Analysis:")
if 'Symbol' in df.columns:
    duplicate_symbols = df['Symbol'].duplicated().sum()
    print(f"Duplicate symbols: {duplicate_symbols}")
    if duplicate_symbols > 0:
        print("🔍 Duplicate symbols found:")
        duplicated_syms = df[df['Symbol'].duplicated(keep=False)]['Symbol'].value_counts()
        print(duplicated_syms)
    else:
        print("✅ All company symbols are unique")
print(f"\n📊 Shape after basic cleaning: {df.shape} (removed {initial_shape[0] - df.shape[0]} rows)")

🧹 STEP 1: BASIC DATA CLEANING
Duplicate rows found: 0

📋 Company Symbol Analysis:
Duplicate symbols: 0
✅ All company symbols are unique

📊 Shape after basic cleaning: (503, 15) (removed 0 rows)


In [52]:
print("🧹 STEP 2: EMPLOYEE DATA STANDARDIZATION")
print("=" * 45)
if 'Full Time Employees' in df.columns:
    print("Processing employee count data...")
    print(f"Original data type: {df['Full Time Employees'].dtype}")
    
    def clean_employee_count(value):
        if pd.isna(value):
            return np.nan
        if isinstance(value, (int, float)):
            return float(value)
        
        value_str = str(value).strip()
        if value_str == '' or value_str.lower() in ['nan', 'null', 'none']:
            return np.nan
        
        value_clean = re.sub(r'[^0-9.]', '', value_str)
        try:
            return float(value_clean)
        except:
            return np.nan
    
    df['Full Time Employees'] = df['Full Time Employees'].apply(clean_employee_count)
    
    employee_stats = df['Full Time Employees'].describe()
    print(f"✅ Employee data standardized:")
    print(f"   Valid records: {df['Full Time Employees'].count()}/{len(df)}")
    print(f"   Range: {employee_stats['min']:,.0f} to {employee_stats['max']:,.0f}")
    print(f"   Median: {employee_stats['50%']:,.0f}")
    
    extreme_outliers = df['Full Time Employees'] > 1000000
    if extreme_outliers.any():
        print(f"\n⚠️  Large companies (>1M employees):")
        large_companies = df[extreme_outliers][['Symbol', 'Name', 'Full Time Employees']]
        for _, row in large_companies.iterrows():
            print(f"   {row['Symbol']}: {row['Full Time Employees']:,.0f} employees")
else:
    print("❌ 'Full Time Employees' column not found")

🧹 STEP 2: EMPLOYEE DATA STANDARDIZATION
Processing employee count data...
Original data type: object
✅ Employee data standardized:
   Valid records: 498/503
   Range: 28 to 2,100,000
   Median: 21,585

⚠️  Large companies (>1M employees):
   WMT: 2,100,000 employees
   AMZN: 1,525,000 employees


In [53]:
print("🧹 STEP 3: ESG RISK SCORES PREPROCESSING")
print("=" * 45)
esg_columns = []
for col in df.columns:
    if any(keyword in col.lower() for keyword in ['esg', 'risk', 'score', 'environment', 'social', 'governance']):
        if df[col].dtype in ['int64', 'float64'] or col in ['Total ESG Risk score', 'Environment Risk Score', 'Governance Risk Score', 'Social Risk Score', 'Controversy Score']:
            esg_columns.append(col)
print(f"🎯 Identified {len(esg_columns)} ESG-related numeric columns:")
for i, col in enumerate(esg_columns, 1):
    print(f"   {i}. {col}")
if esg_columns:
    print(f"\n📊 ESG Data Completeness Analysis:")
    for col in esg_columns:
        valid_count = df[col].count()
        missing_count = df[col].isnull().sum()
        missing_pct = (missing_count / len(df)) * 100
        print(f"   {col:<25}: {valid_count:3d}/{len(df)} valid ({100-missing_pct:5.1f}%)")
    
    print(f"\n🔍 ESG Score Ranges:")
    for col in esg_columns:
        if df[col].count() > 0:
            min_val = df[col].min()
            max_val = df[col].max()
            mean_val = df[col].mean()
            print(f"   {col:<25}: {min_val:6.1f} - {max_val:6.1f} (mean: {mean_val:6.1f})")
else:
    print("❌ No ESG columns identified")

🧹 STEP 3: ESG RISK SCORES PREPROCESSING
🎯 Identified 5 ESG-related numeric columns:
   1. Total ESG Risk score
   2. Environment Risk Score
   3. Governance Risk Score
   4. Social Risk Score
   5. Controversy Score

📊 ESG Data Completeness Analysis:
   Total ESG Risk score     : 430/503 valid ( 85.5%)
   Environment Risk Score   : 430/503 valid ( 85.5%)
   Governance Risk Score    : 430/503 valid ( 85.5%)
   Social Risk Score        : 430/503 valid ( 85.5%)
   Controversy Score        : 403/503 valid ( 80.1%)

🔍 ESG Score Ranges:
   Total ESG Risk score     :    7.1 -   41.7 (mean:   21.5)
   Environment Risk Score   :    0.0 -   25.0 (mean:    5.7)
   Governance Risk Score    :    3.0 -   19.4 (mean:    6.7)
   Social Risk Score        :    0.8 -   22.5 (mean:    9.1)
   Controversy Score        :    1.0 -    5.0 (mean:    2.0)


In [54]:
print("🧹 STEP 4: CATEGORICAL DATA STANDARDIZATION")
print("=" * 45)
categorical_columns = ['Sector', 'Industry', 'Controversy Level', 'ESG Risk Level', 'ESG Risk Percentile']
for col in categorical_columns:
    if col in df.columns:
        print(f"\n📊 Processing {col}:")
        
        df[col] = df[col].astype(str).str.strip()
        df[col] = df[col].replace(['nan', 'None', 'null', ''], np.nan)
        
        unique_values = df[col].value_counts(dropna=False)
        print(f"   Unique values: {len(unique_values)} (including NaN)")
        print(f"   Missing: {df[col].isnull().sum()} ({(df[col].isnull().sum()/len(df)*100):.1f}%)")
        
        if col == 'ESG Risk Percentile':
            print(f"   Sample values: {list(unique_values.head().index)}")
            
            def extract_percentile_number(value):
                if pd.isna(value) or value == 'nan':
                    return np.nan
                try:
                    percentile_match = re.search(r'(\d+)', str(value))
                    if percentile_match:
                        return int(percentile_match.group(1))
                    return np.nan
                except:
                    return np.nan
            
            df['ESG_Risk_Percentile_Numeric'] = df[col].apply(extract_percentile_number)
            valid_percentiles = df['ESG_Risk_Percentile_Numeric'].dropna()
            if len(valid_percentiles) > 0:
                print(f"   ✅ Created numeric percentile column: {valid_percentiles.min():.0f}-{valid_percentiles.max():.0f} range")
        
        elif col in ['Controversy Level', 'ESG Risk Level']:
            print(f"   Top categories: {list(unique_values.head().index)}")
        
        elif col in ['Sector', 'Industry']:
            print(f"   Top 5 categories: {list(unique_values.head().index)}")
    else:
        print(f"   ❌ {col} not found in dataset")

🧹 STEP 4: CATEGORICAL DATA STANDARDIZATION

📊 Processing Sector:
   Unique values: 12 (including NaN)
   Missing: 1 (0.2%)
   Top 5 categories: ['Technology', 'Industrials', 'Financial Services', 'Healthcare', 'Consumer Cyclical']

📊 Processing Industry:
   Unique values: 117 (including NaN)
   Missing: 1 (0.2%)
   Top 5 categories: ['Utilities - Regulated Electric', 'Specialty Industrial Machinery', 'Semiconductors', 'Software - Application', 'Aerospace & Defense']

📊 Processing Controversy Level:
   Unique values: 7 (including NaN)
   Missing: 73 (14.5%)
   Top categories: ['Moderate Controversy Level', 'Low Controversy Level', 'Significant Controversy Level', nan, 'None Controversy Level']

📊 Processing ESG Risk Level:
   Unique values: 6 (including NaN)
   Missing: 73 (14.5%)
   Top categories: ['Low', 'Medium', nan, 'High', 'Negligible']

📊 Processing ESG Risk Percentile:
   Unique values: 90 (including NaN)
   Missing: 73 (14.5%)
   Sample values: [nan, '16th percentile', '23rd p

In [55]:
print("🧹 STEP 5: MISSING VALUE TREATMENT STRATEGY")
print("=" * 50)
missing_analysis = pd.DataFrame({
    'Column': df.columns,
    'Missing_Count': df.isnull().sum(),
    'Missing_Percentage': (df.isnull().sum() / len(df)) * 100,
    'Data_Type': df.dtypes
}).sort_values('Missing_Percentage', ascending=False)
high_missing = missing_analysis[missing_analysis['Missing_Percentage'] > 10]
medium_missing = missing_analysis[(missing_analysis['Missing_Percentage'] > 1) & (missing_analysis['Missing_Percentage'] <= 10)]
low_missing = missing_analysis[(missing_analysis['Missing_Percentage'] > 0) & (missing_analysis['Missing_Percentage'] <= 1)]
print(f"📊 Missing Value Categories:")
print(f"   🔴 High missing (>10%): {len(high_missing)} columns")
print(f"   🟡 Medium missing (1-10%): {len(medium_missing)} columns")
print(f"   🟢 Low missing (<1%): {len(low_missing)} columns")
if len(high_missing) > 0:
    print(f"\n🔴 High Missing Columns:")
    for _, row in high_missing.iterrows():
        print(f"   {row['Column']:<25}: {row['Missing_Percentage']:5.1f}% missing")
imputation_strategy = {}
for col in df.columns:
    missing_pct = (df[col].isnull().sum() / len(df)) * 100
    
    if missing_pct == 0:
        imputation_strategy[col] = 'no_action'
    elif missing_pct > 25:
        imputation_strategy[col] = 'flag_and_median'
    elif missing_pct > 10:
        if col in esg_columns:
            imputation_strategy[col] = 'knn_imputation'
        else:
            imputation_strategy[col] = 'median_mode'
    elif missing_pct > 1:
        imputation_strategy[col] = 'median_mode'
    else:
        imputation_strategy[col] = 'forward_fill'
print(f"\n📋 Imputation Strategy Summary:")
strategy_counts = pd.Series(list(imputation_strategy.values())).value_counts()
for strategy, count in strategy_counts.items():
    print(f"   {strategy}: {count} columns")

🧹 STEP 5: MISSING VALUE TREATMENT STRATEGY
📊 Missing Value Categories:
   🔴 High missing (>10%): 9 columns
   🟡 Medium missing (1-10%): 0 columns
   🟢 Low missing (<1%): 5 columns

🔴 High Missing Columns:
   Controversy Score        :  19.9% missing
   Total ESG Risk score     :  14.5% missing
   Environment Risk Score   :  14.5% missing
   Governance Risk Score    :  14.5% missing
   Social Risk Score        :  14.5% missing
   Controversy Level        :  14.5% missing
   ESG Risk Percentile      :  14.5% missing
   ESG Risk Level           :  14.5% missing
   ESG_Risk_Percentile_Numeric:  14.5% missing

📋 Imputation Strategy Summary:
   forward_fill: 5 columns
   knn_imputation: 5 columns
   median_mode: 4 columns
   no_action: 2 columns


In [56]:
missingness_original = df.isna()
env_cols = [c for c in df.columns if any(k in c.lower() for k in ['environment']) and pd.api.types.is_numeric_dtype(df[c])]
social_cols = [c for c in df.columns if any(k in c.lower() for k in ['social']) and pd.api.types.is_numeric_dtype(df[c])]
gov_cols = [c for c in df.columns if any(k in c.lower() for k in ['governance']) and pd.api.types.is_numeric_dtype(df[c])]
if 'Environment Risk Score' in df.columns and 'Environment Risk Score' not in env_cols and pd.api.types.is_numeric_dtype(df['Environment Risk Score']):
    env_cols.append('Environment Risk Score')
if 'Social Risk Score' in df.columns and 'Social Risk Score' not in social_cols and pd.api.types.is_numeric_dtype(df['Social Risk Score']):
    social_cols.append('Social Risk Score')
if 'Governance Risk Score' in df.columns and 'Governance Risk Score' not in gov_cols and pd.api.types.is_numeric_dtype(df['Governance Risk Score']):
    gov_cols.append('Governance Risk Score')
missing_env_flag = (df[env_cols].isna().any(axis=1).astype(int) if len(env_cols) > 0 else pd.Series(0, index=df.index))
missing_social_flag = (df[social_cols].isna().any(axis=1).astype(int) if len(social_cols) > 0 else pd.Series(0, index=df.index))
missing_gov_flag = (df[gov_cols].isna().any(axis=1).astype(int) if len(gov_cols) > 0 else pd.Series(0, index=df.index))
print(f"missing_env_flag=1: {int(missing_env_flag.sum())}")
print(f"missing_social_flag=1: {int(missing_social_flag.sum())}")
print(f"missing_gov_flag=1: {int(missing_gov_flag.sum())}")

missing_env_flag=1: 73
missing_social_flag=1: 73
missing_gov_flag=1: 73


In [57]:
print("🔧 STEP 6: IMPLEMENTING MISSING VALUE IMPUTATION")
print("=" * 50)
df_imputed = df.copy()
for col, strategy in imputation_strategy.items():
    if strategy == 'no_action':
        continue
    
    missing_before = df_imputed[col].isnull().sum()
    
    if strategy == 'forward_fill':
        if df_imputed[col].dtype in ['object']:
            df_imputed[col].fillna(method='ffill', inplace=True)
            df_imputed[col].fillna('Unknown', inplace=True)
        else:
            df_imputed[col].fillna(df_imputed[col].median(), inplace=True)
    
    elif strategy == 'median_mode':
        if df_imputed[col].dtype in ['object']:
            mode_val = df_imputed[col].mode()
            fill_val = mode_val.iloc[0] if len(mode_val) > 0 else 'Unknown'
            df_imputed[col].fillna(fill_val, inplace=True)
        else:
            df_imputed[col].fillna(df_imputed[col].median(), inplace=True)
    
    elif strategy == 'flag_and_median':
        df_imputed[f'{col}_was_missing'] = df_imputed[col].isnull().astype(int)
        if df_imputed[col].dtype in ['object']:
            mode_val = df_imputed[col].mode()
            fill_val = mode_val.iloc[0] if len(mode_val) > 0 else 'Unknown'
            df_imputed[col].fillna(fill_val, inplace=True)
        else:
            df_imputed[col].fillna(df_imputed[col].median(), inplace=True)
    
    elif strategy == 'knn_imputation' and col in esg_columns:
        if df_imputed[col].count() > 0:
            esg_subset = df_imputed[esg_columns].select_dtypes(include=[np.number])
            if esg_subset.shape[1] > 1:
                try:
                    imputer = KNNImputer(n_neighbors=5)
                    esg_imputed = imputer.fit_transform(esg_subset)
                    
                    col_idx = esg_subset.columns.get_loc(col)
                    df_imputed[col] = esg_imputed[:, col_idx]
                except:
                    df_imputed[col].fillna(df_imputed[col].median(), inplace=True)
            else:
                df_imputed[col].fillna(df_imputed[col].median(), inplace=True)
    
    missing_after = df_imputed[col].isnull().sum()
    
    if missing_before > 0:
        print(f"   {col:<30}: {missing_before:3d} → {missing_after:3d} missing ({strategy})")

print(f"\n✅ Imputation completed!")
print(f"📊 Total missing values: {df.isnull().sum().sum()} → {df_imputed.isnull().sum().sum()}")
print(f"📊 Data completeness: {((df_imputed.shape[0] * df_imputed.shape[1] - df_imputed.isnull().sum().sum()) / (df_imputed.shape[0] * df_imputed.shape[1]) * 100):.1f}%")

🔧 STEP 6: IMPLEMENTING MISSING VALUE IMPUTATION
   Address                       :   1 →   0 missing (forward_fill)
   Sector                        :   1 →   0 missing (forward_fill)
   Industry                      :   1 →   0 missing (forward_fill)
   Full Time Employees           :   5 →   0 missing (forward_fill)
   Description                   :   1 →   0 missing (forward_fill)
   Total ESG Risk score          :  73 →   0 missing (knn_imputation)
   Environment Risk Score        :  73 →   0 missing (knn_imputation)
   Governance Risk Score         :  73 →   0 missing (knn_imputation)
   Social Risk Score             :  73 →   0 missing (knn_imputation)
   Controversy Level             :  73 →   0 missing (median_mode)
   Controversy Score             : 100 →   0 missing (knn_imputation)
   ESG Risk Percentile           :  73 →   0 missing (median_mode)
   ESG Risk Level                :  73 →   0 missing (median_mode)
   ESG_Risk_Percentile_Numeric   :  73 →   0 missing (median_

In [58]:
df_imputed['missing_env_flag'] = missing_env_flag.values
df_imputed['missing_social_flag'] = missing_social_flag.values
df_imputed['missing_gov_flag'] = missing_gov_flag.values
imputed_mask = missingness_original.reindex(df_imputed.index, columns=df_imputed.columns, fill_value=False)
common_cols = [c for c in imputed_mask.columns if c in df_imputed.columns]
imputed_fraction = imputed_mask[common_cols].mean(axis=1)
df_imputed['imputed_fraction'] = imputed_fraction.values
if 'Full Time Employees' in df_imputed.columns:
    employees_before_univariate = df_imputed['Full Time Employees'].copy()
else:
    employees_before_univariate = None
print(f"Mean imputed_fraction: {df_imputed['imputed_fraction'].mean():.4f}")
print(f"Rows with any imputation: {int((df_imputed['imputed_fraction'] > 0).sum())}")

Mean imputed_fraction: 0.0725
Rows with any imputation: 103


In [59]:
print("🔧 STEP 7: OUTLIER DETECTION AND TREATMENT")
print("=" * 45)
numeric_columns = df_imputed.select_dtypes(include=[np.number]).columns
exclude_outlier_cols = [c for c in numeric_columns if str(c).endswith('_flag') or str(c).endswith('_was_missing') or str(c) in ['imputed_fraction', 'univariate_outlier_count', 'univariate_outlier_fraction']]
numeric_columns = [c for c in numeric_columns if c not in exclude_outlier_cols]
outlier_summary = []
for col in numeric_columns:
    if df_imputed[col].count() > 0:
        Q1 = df_imputed[col].quantile(0.25)
        Q3 = df_imputed[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        outliers = df_imputed[(df_imputed[col] < lower_bound) | (df_imputed[col] > upper_bound)]
        outlier_count = len(outliers)
        outlier_percentage = (outlier_count / len(df_imputed)) * 100
        outlier_summary.append({
            'Column': col,
            'Outliers': outlier_count,
            'Percentage': outlier_percentage,
            'Lower_Bound': lower_bound,
            'Upper_Bound': upper_bound
        })
outlier_df = pd.DataFrame(outlier_summary)
outlier_df = outlier_df.sort_values('Percentage', ascending=False)
print("📊 Outlier Analysis:")
for _, row in outlier_df.head(10).iterrows():
    if row['Outliers'] > 0:
        print(f"   {row['Column']:<30}: {row['Outliers']:3d} outliers ({row['Percentage']:4.1f}%)")
print(f"\n🎯 Outlier Treatment Strategy:")
for col in numeric_columns:
    if col in outlier_df[outlier_df['Percentage'] > 5]['Column'].values:
        if 'Employee' in col:
            cap_value = df_imputed[col].quantile(0.99)
            outlier_count = (df_imputed[col] > cap_value).sum()
            df_imputed[col] = np.where(df_imputed[col] > cap_value, cap_value, df_imputed[col])
            print(f"   {col}: Capped {outlier_count} values at 99th percentile ({cap_value:,.0f})")
        elif col in esg_columns:
            outlier_count = len(outlier_df[outlier_df['Column'] == col])
            if outlier_count > 0:
                print(f"   {col}: Keeping {outlier_df[outlier_df['Column']==col]['Outliers'].iloc[0]} outliers (legitimate ESG risk variation)")
        else:
            outlier_info = outlier_df[outlier_df['Column'] == col].iloc[0]
            if outlier_info['Outliers'] > 0:
                print(f"   {col}: Flagged {outlier_info['Outliers']} outliers for review")
print(f"\n✅ Outlier treatment completed!")

🔧 STEP 7: OUTLIER DETECTION AND TREATMENT
📊 Outlier Analysis:
   Controversy Score             : 218 outliers (43.3%)
   Full Time Employees           :  52 outliers (10.3%)
   Governance Risk Score         :  19 outliers ( 3.8%)
   Social Risk Score             :  18 outliers ( 3.6%)
   Environment Risk Score        :   9 outliers ( 1.8%)
   Total ESG Risk score          :   7 outliers ( 1.4%)

🎯 Outlier Treatment Strategy:
   Full Time Employees: Capped 6 values at 99th percentile (439,500)
   Controversy Score: Keeping 218 outliers (legitimate ESG risk variation)

✅ Outlier treatment completed!


In [60]:
numeric_columns_for_outliers = df_imputed.select_dtypes(include=[np.number]).columns
bounds = {}
for _, r in outlier_df.iterrows():
    bounds[r['Column']] = (float(r['Lower_Bound']), float(r['Upper_Bound']))
univariate_outlier_count = pd.Series(0, index=df_imputed.index, dtype=int)
for c in numeric_columns_for_outliers:
    if c in bounds:
        lb, ub = bounds[c]
        v = df_imputed[c]
        o = (v < lb) | (v > ub)
        univariate_outlier_count = univariate_outlier_count + o.fillna(False).astype(int)
df_imputed['univariate_outlier_count'] = univariate_outlier_count.values
df_imputed['univariate_outlier_fraction'] = (df_imputed['univariate_outlier_count'] / max(1, len(bounds))).astype(float)
print(f"Mean univariate_outlier_count: {df_imputed['univariate_outlier_count'].mean():.3f}")
print(f"Rows with any univariate outlier: {int((df_imputed['univariate_outlier_count'] > 0).sum())}")

Mean univariate_outlier_count: 0.642
Rows with any univariate outlier: 254


In [61]:
multivar_cols = [c for c in df_imputed.select_dtypes(include=[np.number]).columns if c not in ['univariate_outlier_count', 'univariate_outlier_fraction']]
multivar_cols = [c for c in multivar_cols if not str(c).endswith('_was_missing')]
X_mv = _safe_numeric_frame(df_imputed, multivar_cols)
if X_mv.shape[1] >= 2 and X_mv.shape[0] >= 10:
    iso = IsolationForest(n_estimators=300, contamination='auto', random_state=42)
    iso.fit(X_mv.values)
    iso_score = -iso.score_samples(X_mv.values)
    df_imputed['mv_isolation_forest_score'] = iso_score
    lof = LocalOutlierFactor(n_neighbors=min(35, max(5, int(np.sqrt(X_mv.shape[0])))), contamination='auto')
    lof_fit = lof.fit_predict(X_mv.values)
    lof_score = -lof.negative_outlier_factor_
    df_imputed['mv_lof_score'] = lof_score
    df_imputed['mv_mahalanobis'] = mahalanobis_distance(df_imputed, multivar_cols).values
    iso_thr = float(np.nanquantile(df_imputed['mv_isolation_forest_score'], 0.98))
    lof_thr = float(np.nanquantile(df_imputed['mv_lof_score'], 0.98))
    mah_thr = float(np.nanquantile(df_imputed['mv_mahalanobis'], 0.98))
    df_imputed['mv_outlier_flag'] = ((df_imputed['mv_isolation_forest_score'] >= iso_thr) | (df_imputed['mv_lof_score'] >= lof_thr) | (df_imputed['mv_mahalanobis'] >= mah_thr)).astype(int)
else:
    df_imputed['mv_isolation_forest_score'] = np.nan
    df_imputed['mv_lof_score'] = np.nan
    df_imputed['mv_mahalanobis'] = np.nan
    df_imputed['mv_outlier_flag'] = 0
outlier_penalty = 0.5 * df_imputed['mv_outlier_flag'].astype(float) + 0.5 * (df_imputed['univariate_outlier_count'] > 0).astype(float)
df_imputed['data_confidence'] = (1.0 - (df_imputed['imputed_fraction'].astype(float) + outlier_penalty)).clip(0, 1)
print(f"mv_outlier_flag=1: {int(df_imputed['mv_outlier_flag'].sum())}")
print(f"Mean data_confidence: {df_imputed['data_confidence'].mean():.4f}")
print(f"Low-confidence (<0.7): {int((df_imputed['data_confidence'] < 0.7).sum())}")

mv_outlier_flag=1: 28
Mean data_confidence: 0.6505
Low-confidence (<0.7): 326


In [62]:
quality_cols = [c for c in df.columns if c in df_imputed.columns]
col_quality = []
for c in quality_cols:
    original_fraction = 1.0 - float(missingness_original[c].mean()) if c in missingness_original.columns else np.nan
    imputed_fraction_col = float(missingness_original[c].mean()) if c in missingness_original.columns else np.nan
    outlier_fraction_col = np.nan
    if c in bounds:
        lb, ub = bounds[c]
        v = pd.to_numeric(df_imputed[c], errors='coerce')
        outlier_fraction_col = float(((v < lb) | (v > ub)).mean())
    col_quality.append({'column': c, 'original_fraction': original_fraction, 'imputed_fraction': imputed_fraction_col, 'univariate_outlier_fraction': outlier_fraction_col})
col_quality_df = pd.DataFrame(col_quality).sort_values(['imputed_fraction', 'univariate_outlier_fraction'], ascending=False)
display(col_quality_df.head(20))

,column,original_fraction,imputed_fraction,univariate_outlier_fraction
12,Controversy Score,0.801193,0.198807,0.433400
9,Governance Risk Score,0.854871,0.145129,0.037773
10,Social Risk Score,0.854871,0.145129,0.035785
8,Environment Risk Score,0.854871,0.145129,0.017893
7,Total ESG Risk score,0.854871,0.145129,0.013917
15,ESG_Risk_Percentile_Numeric,0.854871,0.145129,0.000000
11,Controversy Level,0.854871,0.145129,NaN
13,ESG Risk Percentile,0.854871,0.145129,NaN
14,ESG Risk Level,0.854871,0.145129,NaN
5,Full Time Employees,0.990060,0.009940,0.103380


In [63]:
print("🔧 STEP 8: FEATURE ENGINEERING")
print("=" * 35)
df_featured = df_imputed.copy()
if len([col for col in esg_columns if col in df_featured.columns]) >= 3:
    if all(col in df_featured.columns for col in ['Environment Risk Score', 'Social Risk Score', 'Governance Risk Score']):
        df_featured['ESG_Component_Balance'] = df_featured[['Environment Risk Score', 'Social Risk Score', 'Governance Risk Score']].std(axis=1)
        print("✅ Created ESG_Component_Balance (std of E, S, G scores)")
        
        df_featured['ESG_Max_Component'] = df_featured[['Environment Risk Score', 'Social Risk Score', 'Governance Risk Score']].max(axis=1)
        df_featured['ESG_Min_Component'] = df_featured[['Environment Risk Score', 'Social Risk Score', 'Governance Risk Score']].min(axis=1)
        print("✅ Created ESG_Max_Component and ESG_Min_Component")
if 'Full Time Employees' in df_featured.columns:
    df_featured['Employee_Size_Category'] = pd.cut(
        df_featured['Full Time Employees'], 
        bins=[0, 1000, 10000, 50000, float('inf')], 
        labels=['Small', 'Medium', 'Large', 'Enterprise'],
        include_lowest=True
    )
    print("✅ Created Employee_Size_Category (Small/Medium/Large/Enterprise)")
    
    df_featured['Log_Employees'] = np.log1p(df_featured['Full Time Employees'])
    print("✅ Created Log_Employees (log-transformed employee count)")
if 'Total ESG Risk score' in df_featured.columns:
    esg_risk_mean = df_featured['Total ESG Risk score'].mean()
    df_featured['ESG_Risk_Above_Average'] = (df_featured['Total ESG Risk score'] > esg_risk_mean).astype(int)
    print(f"✅ Created ESG_Risk_Above_Average (threshold: {esg_risk_mean:.1f})")
    
    df_featured['ESG_Risk_Category'] = pd.cut(
        df_featured['Total ESG Risk score'],
        bins=[0, 15, 25, 35, float('inf')],
        labels=['Low', 'Medium', 'High', 'Severe'],
        include_lowest=True
    )
    print("✅ Created ESG_Risk_Category (Low/Medium/High/Severe)")
if 'Sector' in df_featured.columns:
    sector_risk_avg = df_featured.groupby('Sector')['Total ESG Risk score'].mean()
    df_featured['Sector_Risk_Average'] = df_featured['Sector'].map(sector_risk_avg)
    df_featured['ESG_vs_Sector_Average'] = df_featured['Total ESG Risk score'] - df_featured['Sector_Risk_Average']
    print("✅ Created Sector_Risk_Average and ESG_vs_Sector_Average")
if 'ESG Risk Percentile' in df_featured.columns and 'ESG_Risk_Percentile_Numeric' in df_featured.columns:
    df_featured['High_Risk_Percentile'] = (df_featured['ESG_Risk_Percentile_Numeric'] > 75).astype(int)
    print("✅ Created High_Risk_Percentile (>75th percentile flag)")
new_features = [col for col in df_featured.columns if col not in df_imputed.columns]
print(f"\n📊 Feature Engineering Summary:")
print(f"   Original features: {len(df_imputed.columns)}")
print(f"   New features: {len(new_features)}")
print(f"   Total features: {len(df_featured.columns)}")
if new_features:
    print(f"\n🆕 New features created:")
    for i, feature in enumerate(new_features, 1):
        print(f"   {i}. {feature}")

🔧 STEP 8: FEATURE ENGINEERING
✅ Created ESG_Component_Balance (std of E, S, G scores)
✅ Created ESG_Max_Component and ESG_Min_Component
✅ Created Employee_Size_Category (Small/Medium/Large/Enterprise)
✅ Created Log_Employees (log-transformed employee count)
✅ Created ESG_Risk_Above_Average (threshold: 21.5)
✅ Created ESG_Risk_Category (Low/Medium/High/Severe)
✅ Created Sector_Risk_Average and ESG_vs_Sector_Average
✅ Created High_Risk_Percentile (>75th percentile flag)

📊 Feature Engineering Summary:
   Original features: 27
   New features: 10
   Total features: 37

🆕 New features created:
   1. ESG_Component_Balance
   2. ESG_Max_Component
   3. ESG_Min_Component
   4. Employee_Size_Category
   5. Log_Employees
   6. ESG_Risk_Above_Average
   7. ESG_Risk_Category
   8. Sector_Risk_Average
   9. ESG_vs_Sector_Average
   10. High_Risk_Percentile


In [64]:
print("🔧 STEP 9: ENCODING CATEGORICAL VARIABLES")
print("=" * 45)
df_encoded = df_featured.copy()
categorical_cols = df_encoded.select_dtypes(include=['object', 'category']).columns
categorical_cols = [col for col in categorical_cols if col not in ['Symbol', 'Name', 'Address', 'Description']]
print(f"📊 Categorical columns to encode: {len(categorical_cols)}")
if len(categorical_cols) > 0:
    for col in categorical_cols:
        print(f"\n🔤 Processing {col}:")
        unique_count = df_encoded[col].nunique()
        print(f"   Unique values: {unique_count}")
        
        if unique_count <= 10:
            dummies = pd.get_dummies(df_encoded[col], prefix=f'{col}', dummy_na=True)
            df_encoded = pd.concat([df_encoded, dummies], axis=1)
            print(f"   ✅ One-hot encoded into {len(dummies.columns)} columns")
        else:
            le = LabelEncoder()
            df_encoded[f'{col}_Encoded'] = le.fit_transform(df_encoded[col].fillna('Unknown'))
            print(f"   ✅ Label encoded as {col}_Encoded")
        
        print(f"   📝 Keeping original {col} for reference")
print(f"\n📊 Encoding Summary:")
print(f"   Columns after encoding: {len(df_encoded.columns)}")
new_encoded_cols = [col for col in df_encoded.columns if col not in df_featured.columns]
print(f"   New encoded columns: {len(new_encoded_cols)}")

🔧 STEP 9: ENCODING CATEGORICAL VARIABLES
📊 Categorical columns to encode: 7

🔤 Processing Sector:
   Unique values: 11
   ✅ Label encoded as Sector_Encoded
   📝 Keeping original Sector for reference

🔤 Processing Industry:
   Unique values: 116
   ✅ Label encoded as Industry_Encoded
   📝 Keeping original Industry for reference

🔤 Processing Controversy Level:
   Unique values: 6
   ✅ One-hot encoded into 7 columns
   📝 Keeping original Controversy Level for reference

🔤 Processing ESG Risk Percentile:
   Unique values: 89
   ✅ Label encoded as ESG Risk Percentile_Encoded
   📝 Keeping original ESG Risk Percentile for reference

🔤 Processing ESG Risk Level:
   Unique values: 5
   ✅ One-hot encoded into 6 columns
   📝 Keeping original ESG Risk Level for reference

🔤 Processing Employee_Size_Category:
   Unique values: 4
   ✅ One-hot encoded into 5 columns
   📝 Keeping original Employee_Size_Category for reference

🔤 Processing ESG_Risk_Category:
   Unique values: 4
   ✅ One-hot encoded in

In [65]:
numeric_cols = df_encoded.select_dtypes(include=[np.number]).columns.tolist()
excluded = [c for c in numeric_cols if str(c).endswith('_was_missing') or str(c).endswith('_flag') or str(c).startswith('prov_')]
candidates = [c for c in numeric_cols if c not in excluded]
drop_corr = correlation_prune(df_encoded, candidates, threshold=0.95)
vif_table = compute_vif(df_encoded, candidates, max_features=40)
print(f"Correlation-pruned candidates to drop: {len(drop_corr)}")
if len(drop_corr) > 0:
    print(drop_corr[:30])
display(vif_table.head(20))
df_encoded_reduced = df_encoded.drop(columns=drop_corr, errors='ignore')
print(f"Columns: {df_encoded.shape[1]} -> {df_encoded_reduced.shape[1]}")

Correlation-pruned candidates to drop: 2
['ESG_Risk_Percentile_Numeric', 'univariate_outlier_fraction']


,feature,vif
0,univariate_outlier_fraction,1.000000e+12
1,Sector_Risk_Average,1.000000e+12
2,ESG_Risk_Category_Medium,1.000000e+12
3,Employee_Size_Category_Enterprise,1.000000e+12
4,ESG_Risk_Category_High,1.000000e+12
5,Employee_Size_Category_Medium,1.000000e+12
6,Employee_Size_Category_Large,1.000000e+12
7,ESG_Risk_Category_Low,1.000000e+12
8,ESG_vs_Sector_Average,1.000000e+12
9,Total ESG Risk score,1.000000e+12


Columns: 63 -> 61


In [66]:
print("🔧 STEP 10: FEATURE SCALING AND NORMALIZATION")
print("=" * 50)
df_final = (df_encoded_reduced.copy() if 'df_encoded_reduced' in globals() else df_encoded.copy())
numeric_cols_to_scale = df_final.select_dtypes(include=[np.number]).columns
exclude_from_scaling = ['Symbol', 'Name', 'Address', 'Description'] +                       [col for col in df_final.columns if '_was_missing' in col or                        col.endswith('_Encoded') or                        col.endswith('_flag') or                        col.startswith('prov_') or                        col.startswith(tuple(['Sector_', 'Industry_', 'Controversy Level_', 'ESG Risk Level_']))]
scale_cols = [col for col in numeric_cols_to_scale if col not in exclude_from_scaling]
print(f"📊 Columns to scale: {len(scale_cols)}")
if len(scale_cols) > 0:
    if 'Sector' in df_final.columns:
        df_final_scaled = df_final.copy()
        scaler_global = StandardScaler()
        sector_values = df_final['Sector'].astype(str)
        for sector in sector_values.unique():
            idx = sector_values == sector
            if idx.sum() < 5:
                continue
            scaler = StandardScaler()
            df_final_scaled.loc[idx, scale_cols] = scaler.fit_transform(df_final.loc[idx, scale_cols])
        missing_scaled = df_final_scaled[scale_cols].isna().any(axis=1)
        if missing_scaled.any():
            df_final_scaled.loc[missing_scaled, scale_cols] = scaler_global.fit_transform(df_final.loc[missing_scaled, scale_cols])
        df_final = df_final_scaled
        print("✅ Sector-wise standardization applied")
    else:
        scaler = StandardScaler()
        df_final[scale_cols] = scaler.fit_transform(df_final[scale_cols])
        print(f"✅ Standardized {len(scale_cols)} numeric columns")
else:
    print("❌ No columns found for scaling")

🔧 STEP 10: FEATURE SCALING AND NORMALIZATION
📊 Columns to scale: 29
✅ Sector-wise standardization applied


In [67]:
numeric_cols_drift = [c for c in df_final.select_dtypes(include=[np.number]).columns if c not in ['univariate_outlier_count', 'univariate_outlier_fraction']]
rng = np.random.RandomState(42)
mask = rng.rand(len(df_final)) < 0.5
df_a = df_final.loc[mask]
df_b = df_final.loc[~mask]
drift_rows = []
for c in numeric_cols_drift:
    a = pd.to_numeric(df_a[c], errors='coerce')
    b = pd.to_numeric(df_b[c], errors='coerce')
    if a.notna().sum() < 10 or b.notna().sum() < 10:
        continue
    ks = ks_2samp(a.dropna().values, b.dropna().values)
    psi = psi_numeric(a, b, bins=10)
    drift_rows.append({'feature': c, 'ks_stat': float(ks.statistic), 'ks_pvalue': float(ks.pvalue), 'psi': psi})
drift_df = pd.DataFrame(drift_rows)
if len(drift_df) > 0:
    drift_df = drift_df.sort_values(['psi', 'ks_stat'], ascending=False).reset_index(drop=True)
    display(drift_df.head(25))
    print(f"Features with PSI>0.2: {int((drift_df['psi'] > 0.2).sum())}")
else:
    print("No drift metrics computed")

,feature,ks_stat,ks_pvalue,psi
0,ESG_Component_Balance,0.061398,0.698910,0.085431
1,ESG Risk Percentile_Encoded,0.082898,0.329258,0.079513
2,Log_Employees,0.072385,0.495138,0.078437
3,ESG_Max_Component,0.051202,0.873270,0.076952
4,Employee_Size_Category_Large,0.066986,0.593328,0.058664
5,ESG_Risk_Category_Medium,0.081790,0.344671,0.058049
6,mv_isolation_forest_score,0.065989,0.612143,0.057652
7,ESG_Min_Component,0.071499,0.510706,0.056889
8,Total ESG Risk score,0.082075,0.340739,0.049737
9,ESG_vs_Sector_Average,0.082075,0.340739,0.049737


Features with PSI>0.2: 0


In [68]:
print("📊 FINAL DATA QUALITY ASSESSMENT")
print("=" * 45)
print(f"🎯 Processing Summary:")
print(f"   Original shape: {df_raw.shape}")
print(f"   Final shape: {df_final.shape}")
print(f"   Rows added/removed: {df_final.shape[0] - df_raw.shape[0]:+d}")
print(f"   Columns added: {df_final.shape[1] - df_raw.shape[1]:+d}")
print(f"\n📈 Data Quality Metrics:")
original_completeness = ((df_raw.shape[0] * df_raw.shape[1] - df_raw.isnull().sum().sum()) / (df_raw.shape[0] * df_raw.shape[1]) * 100)
final_completeness = ((df_final.shape[0] * df_final.shape[1] - df_final.isnull().sum().sum()) / (df_final.shape[0] * df_final.shape[1]) * 100)
print(f"   Original completeness: {original_completeness:.1f}%")
print(f"   Final completeness: {final_completeness:.1f}%")
print(f"   Improvement: {final_completeness - original_completeness:+.1f}%")
print(f"\n🔢 Column Types:")
numeric_count = len(df_final.select_dtypes(include=[np.number]).columns)
categorical_count = len(df_final.select_dtypes(include=['object', 'category']).columns)
print(f"   Numeric columns: {numeric_count}")
print(f"   Categorical columns: {categorical_count}")
print(f"   Total columns: {len(df_final.columns)}")
remaining_missing = df_final.isnull().sum().sum()
if remaining_missing > 0:
    print(f"\n⚠️  Remaining missing values: {remaining_missing}")
    missing_cols = df_final.columns[df_final.isnull().any()].tolist()
    for col in missing_cols[:5]:
        missing_count = df_final[col].isnull().sum()
        print(f"   {col}: {missing_count} missing")
else:
    print(f"\n✅ No missing values remaining!")
print(f"\n🎯 Ready for Machine Learning:")
ml_ready_cols = [col for col in df_final.columns 
                if col not in ['Symbol', 'Name', 'Address', 'Description'] 
                and df_final[col].dtype in [np.number, 'int64', 'float64']]
print(f"   ML-ready numeric features: {len(ml_ready_cols)}")
print(f"   Memory usage: {df_final.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

📊 FINAL DATA QUALITY ASSESSMENT
🎯 Processing Summary:
   Original shape: (503, 15)
   Final shape: (503, 61)
   Rows added/removed: +0
   Columns added: +46

📈 Data Quality Metrics:
   Original completeness: 91.8%
   Final completeness: 100.0%
   Improvement: +8.2%

🔢 Column Types:
   Numeric columns: 50
   Categorical columns: 11
   Total columns: 61

✅ No missing values remaining!

🎯 Ready for Machine Learning:
   ML-ready numeric features: 30
   Memory usage: 1.14 MB


In [69]:
print("💾 SAVING PROCESSED DATA")
print("=" * 30)
os.makedirs(output_path, exist_ok=True)
processed_file_path = os.path.join(output_path, 'processed.csv')
metadata_file_path = os.path.join(output_path, 'processing_metadata.txt')
try:
    df_final.to_csv(processed_file_path, index=False)
    print(f"✅ Processed data saved to: {processed_file_path}")
    print(f"📊 File size: {os.path.getsize(processed_file_path) / 1024**2:.2f} MB")
    
    metadata = f"""ESG SUSTAINABILITY DATA PROCESSING METADATA
Generated on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
====================================================

ORIGINAL DATA:
- Shape: {df_raw.shape}
- Completeness: {original_completeness:.1f}%
- Missing values: {df_raw.isnull().sum().sum():,}

PROCESSED DATA:
- Shape: {df_final.shape}
- Completeness: {final_completeness:.1f}%
- Missing values: {df_final.isnull().sum().sum():,}
- ML-ready features: {len(ml_ready_cols)}

PROCESSING STEPS:
1. ✅ Basic cleaning (duplicates, symbols)
2. ✅ Employee data standardization
3. ✅ ESG scores preprocessing
4. ✅ Categorical data standardization
5. ✅ Missing value imputation
6. ✅ Outlier detection and treatment
7. ✅ Feature engineering
8. ✅ Categorical encoding
9. ✅ Feature scaling/normalization
10. ✅ Final quality assessment

NEW FEATURES CREATED:
{chr(10).join([f"- {feature}" for feature in new_features])}

ENCODED COLUMNS:
{chr(10).join([f"- {col}" for col in new_encoded_cols])}

IMPUTATION STRATEGIES USED:
{chr(10).join([f"- {strategy}: {count} columns" for strategy, count in strategy_counts.items()])}

READY FOR:
- Machine Learning model training
- Statistical analysis
- Predictive modeling
- ESG risk assessment
"""

    with open(metadata_file_path, 'w', encoding='utf-8') as f:
        f.write(metadata)
    
    print(f"✅ Processing metadata saved to: {metadata_file_path}")
    
except Exception as e:
    print(f"❌ Error saving files: {e}")

print(f"\n🎉 DATA PREPROCESSING COMPLETED SUCCESSFULLY!")
print(f"📈 Dataset ready for machine learning and advanced analytics")
print(f"📁 Processed files location: {output_path}")
print(f"⏱️  Processing completed at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

💾 SAVING PROCESSED DATA
✅ Processed data saved to: ../data/processed/processed.csv
📊 File size: 1.04 MB
✅ Processing metadata saved to: ../data/processed/processing_metadata.txt

🎉 DATA PREPROCESSING COMPLETED SUCCESSFULLY!
📈 Dataset ready for machine learning and advanced analytics
📁 Processed files location: ../data/processed/
⏱️  Processing completed at: 2026-01-28 23:39:46


In [70]:
print("🔍 PROCESSED DATA SAMPLE PREVIEW")
print("=" * 40)
print(f"📋 Final Dataset Shape: {df_final.shape}")
print(f"\n📊 First 3 rows of key columns:")
key_columns = ['Symbol', 'Name', 'Sector', 'Total ESG Risk score', 'ESG_Risk_Category', 'Employee_Size_Category']
display_cols = [col for col in key_columns if col in df_final.columns]
if len(display_cols) > 0:
    display(df_final[display_cols].head(3))
print(f"\n📈 Statistical Summary of Key ESG Metrics:")
esg_metrics = [col for col in df_final.columns if any(kw in col.lower() for kw in ['esg', 'risk', 'score']) and df_final[col].dtype in [np.number]]
if len(esg_metrics) > 0:
    display(df_final[esg_metrics[:6]].describe().round(3))
print(f"\n🎯 Column Categories:")
print(f"   📊 Numeric: {len(df_final.select_dtypes(include=[np.number]).columns)} columns")
print(f"   🔤 Categorical: {len(df_final.select_dtypes(include=['object']).columns)} columns")
print(f"   🆕 Engineered: {len(new_features)} columns")
print(f"   🎨 Encoded: {len(new_encoded_cols)} columns")
print(f"\n✅ Data preprocessing pipeline completed successfully!")
print(f"🚀 Ready for model training and advanced analytics")

🔍 PROCESSED DATA SAMPLE PREVIEW
📋 Final Dataset Shape: (503, 61)

📊 First 3 rows of key columns:


,Symbol,Name,Sector,Total ESG Risk score,ESG_Risk_Category,Employee_Size_Category
0,ENPH,"Enphase Energy, Inc.",Technology,0.770526,Medium,Medium
1,EMN,Eastman Chemical Company,Basic Materials,-0.133872,High,Large
2,DPZ,Domino's Pizza Inc.,Consumer Cyclical,1.759907,High,Medium



📈 Statistical Summary of Key ESG Metrics:


,Total ESG Risk score,Environment Risk Score,Governance Risk Score,Social Risk Score,Controversy Score,mv_isolation_forest_score
count,503.000,503.000,503.000,503.000,503.000,503.000
mean,0.000,-0.000,0.000,-0.000,-0.000,-0.000
std,1.001,1.001,1.001,1.001,1.001,1.001
min,-2.588,-2.289,-2.103,-2.389,-2.414,-1.987
25%,-0.718,-0.743,-0.766,-0.671,-0.605,-0.743
50%,-0.001,-0.105,-0.111,-0.089,-0.084,-0.311
75%,0.651,0.618,0.575,0.561,0.546,0.608
max,3.534,3.960,3.798,3.509,3.867,4.911



🎯 Column Categories:
   📊 Numeric: 50 columns
   🔤 Categorical: 9 columns
   🆕 Engineered: 10 columns
   🎨 Encoded: 26 columns

✅ Data preprocessing pipeline completed successfully!
🚀 Ready for model training and advanced analytics
